In [109]:
import pandas as pd
import seaborn as sns
from datetime import date

In [110]:
def find_sheet_name(sheet_names):
    filtered_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower() and "dịch vụ" not in x.lower())
            and "nk" in x.lower()
            and ("thang" in x.lower() or "tháng" in x.lower()),
            sheet_names,
        )
    )

    if len(filtered_sheet_names) == 1:
        return filtered_sheet_names[0]

    filtered_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower() and "dịch vụ" not in x.lower())
            and ("nhập khẩu" in x.lower() or "nhap khau" in x.lower())
            and ("thang" in x.lower() or "tháng" in x.lower()),
            sheet_names,
        )
    )

    if len(filtered_sheet_names) == 1:
        return filtered_sheet_names[0]

    filtered_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower() and "dịch vụ" not in x.lower())
            and "nk" in x.lower(),
            sheet_names,
        )
    )

    if len(filtered_sheet_names) == 1:
        return filtered_sheet_names[0]

    filtered_sheet_names = list(
        filter(
            lambda x: ("gia" not in x.lower() and "csg" not in x.lower() and "dịch vụ" not in x.lower())
            and ("nhập khẩu" in x.lower() or "nhap khau" in x.lower()),
            sheet_names,
        )
    )

    if len(filtered_sheet_names) == 1:
        return filtered_sheet_names[0]

    if len(sheet_names) == 43:
        return "27"

    raise RuntimeError(sheet_names, filtered_sheet_names)

In [111]:
en_to_vn = {
    "TOTAL VALUE": "TỔNG TRỊ GIÁ",
    "Domestic economic sector": "Khu vực kinh tế trong nước",
    "FDI sector": "Khu vực có vốn đầu tư NN",
    "MAIN ITEMS": "MẶT HÀNG CHỦ YẾU",
    "Aquatic products": "Thủy sản",
    "Milk and dairy products": "Sữa và sản phẩm sữa",
    "Vegetables and fruits": "Rau quả",
    "Wheat": "Lúa mỳ",
    "Maize": "Ngô",
    "Cattle feed and supplies": "Thức ăn gia súc và NPL",
    "Coal": "Than đá",
    "Crude oil": "Dầu thô",
    "Petroleum": "Xăng dầu",
    "Chemicals": "Hóa chất",
    "Chemical products": "Sản phẩm hoá chất",
    "Medicament": "Tân dược",
    "Fertilizer": "Phân bón",
    "Plastic": "Chất dẻo",
    "Plastic products": "Sản phẩm chất dẻo",
    "Rubber": "Cao su",
    "Wood and products": "Gỗ và sản phẩm gỗ",
    "Paper of all kinds": "Giấy các loại",
    "Cotton": "Bông",
    "Textile yarn": "Sợi dệt",
    "Fabrics": "Vải",
    "Auxiliary materials for textile, footgear": "Nguyên PL dệt, may, giày dép",
    "Glass and glass products": "Thủy tinh và các sản phẩm từ thủy tinh",
    "Iron and steel scrap": "Phế liệu sắt thép",
    "Iron, steel": "Sắt thép",
    "Iron, steel products": "Sản phẩm từ sắt thép",
    "Other basic metals": "Kim loại thường khác",
    "Products made from other basic metals": "Sản phẩm từ kim loại thường khác",
    "Electronic devices, computers and their parts": "Điện tử, máy tính và linh kiện",
    "Domestic electrical appliances and components": "Hàng điện gia dụng và linh kiện",
    "Phones all of kinds and their parts": "Điện thoại và linh kiện",
    "Cameras, camcorders and their components": "Máy ảnh, máy quay phim và linh kiện",
    "Machinery, instrument, accessory": "Máy móc thiết bị, dụng cụ PT khác",
    "Electrical wire and cable": "Dây điện và cáp điện",
    "Automobile": "Ô tô",
    "Of which: Assembled(*)": "Trong đó: Nguyên chiếc(*)",
    "Other means of transport and components": "Phương tiện vận tải khác và PT",
    "(*)Unit, US$ million": "(*)Chiếc, triệu USD",
}

In [113]:
def get_nk(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df = df.dropna(axis=1, how="all")

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Khu vực kinh tế trong nước")
                | (df[df.columns[1]] == "Thủy sản")
                | (df[df.columns[1]] == "Aquatic products")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
    )

    df[df.columns[0]] = df[df.columns[0]].str.strip()

    df = df.reset_index(drop=True)

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG TRỊ GIÁ")
                    | ((df[df.columns[0]] == "TOTAL VALUE"))
                    | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ (*)"))
                    | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ(*)"))
                ].index[0]
            )
        )
    )

    df = df.reset_index(drop=True)

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL VALUE"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    if len(df.columns) > 12:
        df = df.drop(
            columns=[
                df.columns[idx_column] for idx_column in range(12, len(df.columns))
            ]
        )

    total_row = df[df.columns[0]][
        (df[df.columns[0]] == "TỔNG TRỊ GIÁ")
        | ((df[df.columns[0]] == "TOTAL VALUE"))
        | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ (*)"))
        | ((df[df.columns[0]] == "TỔNG TRỊ GIÁ(*)"))
    ].index[0]

    columns_is_nan = [
        df.columns[idx_column]
        for idx_column, is_nan in enumerate(df.loc[total_row].isnull())
        if is_nan == True
    ]

    df = df.drop(columns=columns_is_nan)

    if len(df.columns) > 5:
        df = df.drop(
            columns=[df.columns[idx_column] for idx_column in range(5, len(df.columns))]
        )

    if int(date.split("-")[0]) > 2018 or (
        int(date.split("-")[0]) == 2018 and int(date.split("-")[1]) > 8
    ):
        if len(df.columns) == 5 and int(date.split("-")[1]) == 1:
            df = df.drop(columns=[df.columns[1], df.columns[4]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif len(df.columns) == 5:
            df.columns = ["Ngành", "Total-M", "Total-YOY", "YOY", "YTD-YOY"]
        elif len(df.columns) == 4 and int(date.split("-")[1]) == 1:
            df = df.drop(columns=[df.columns[1]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        assert len(df.columns) == 5, df.columns

        df = df[df["Total-M"].notna()].replace("-", None)
        df = df[df["Total-YOY"].notna()].replace("-", None)
        df = df[df["YOY"].notna()].replace("-", None)
        df = df[df["YTD-YOY"].notna()].replace("-", None)

        df["YOY"] = df["YOY"] - 100
        df["YTD-YOY"] = df["YTD-YOY"] - 100
    else:
        df = df.drop(columns=[df.columns[1]])

        if int(date.split("-")[1]) == 1 and len(df.columns) == 4:
            df = df.drop(columns=[df.columns[3]])
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif int(date.split("-")[1]) == 1 and len(df.columns) == 3:
            df.columns = ["Ngành", "Total-M", "YOY"]
            df["Total-YOY"] = df["Total-M"]
            df["YTD-YOY"] = df["YOY"]
        elif len(df.columns) == 4:
            df.columns = ["Ngành", "Total-M", "Total-YOY", "YTD-YOY"]
        else:
            raise RuntimeError

        df = df[df["Total-M"].notna()].replace("-", None)
        df = df[df["Total-YOY"].notna()].replace("-", None)
        df = df[df["YTD-YOY"].notna()].replace("-", None)

        df["YOY"] = df["YOY"].replace("-", None) - 100
        df["YTD-YOY"] = df["YTD-YOY"].replace("-", None) - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    return df


# xl = pd.ExcelFile("datas/2018-08-29-02-Bieu-8.2018-1.xlsx")

# get_nk(xl, find_sheet_name(xl.sheet_names), "2018-08-29")

In [114]:
import os
import contextlib

dfs_nk_origin = []
for file_name in sorted(
    os.listdir("./datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("./datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2019:
            continue

        xl = pd.ExcelFile(file_path)

        df_xk = get_nk(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_nk_origin.append(df_xk)
    except Exception as e:
        print(file_name)
        raise e

In [115]:
dfs_nk = pd.concat(dfs_nk_origin, ignore_index=True, axis=0)
dfs_nk["Date"] = pd.to_datetime(dfs_nk["Date"], dayfirst=False)
dfs_nk

,Ngành,Total-M,Total-YOY,YOY,YTD-YOY,Date
0,TỔNG TRỊ GIÁ,31080,84980,9.70548,13.883591,2024-03-29
1,Khu vực kinh tế trong nước,10830,29696.990639,2.912738,14.393183,2024-03-29
2,Khu vực có vốn đầu tư NN,20250,55283.009361,13.719834,13.611718,2024-03-29
3,Thủy sản,230,636.436115,-6.206037,-3.933351,2024-03-29
4,Sữa và sản phẩm sữa,90,236.389268,-26.184634,-25.339836,2024-03-29
...,...,...,...,...,...,...
2477,"Máy móc thiết bị, DC, PT khác",3000,3000,3.786542,3.786542,2019-01-29
2478,Ô tô,597.929161,597.929161,118.872049,118.872049,2019-01-29
2479,Trong đó: Nguyên chiếc(*),247.929161,247.929161,1263.290828,1263.290828,2019-01-29
2480,"Xe máy và linh kiện, phụ tùng",75,75,53.731541,53.731541,2019-01-29


In [116]:
convert_nganh = {
    "TỔNG TRỊ GIÁ": "TỔNG TRỊ GIÁ",
    "Khu vực kinh tế trong nước": "Khu vực kinh tế trong nước",
    "Khu vực có vốn đầu tư NN": "Khu vực có vốn đầu tư NN",
    "Thủy sản": "Thủy sản",
    "Sữa và sản phẩm sữa": "Sữa và sản phẩm sữa",
    "Rau quả": "Rau quả",
    "Hạt điều": "Hạt điều",
    "Ngô": "Ngô",
    "Thức ăn gia súc và NPL": "Thức ăn gia súc và NPL",
    "Quặng và khoáng sản khác": "Quặng và khoáng sản khác",
    "Than đá": "Than đá",
    "Dầu thô": "Dầu thô",
    "Xăng dầu": "Xăng dầu",
    "Khí đốt hóa lỏng": "Khí đốt hóa lỏng",
    "Hóa chất": "Hóa chất",
    "Sản phẩm hoá chất": "Sản phẩm hoá chất",
    "Tân dược": "Tân dược",
    "Phân bón": "Phân bón",
    "Chất dẻo": "Chất dẻo",
    "Sản phẩm chất dẻo": "Sản phẩm chất dẻo",
    "Cao su": "Cao su",
    "Gỗ và sản phẩm gỗ": "Gỗ và sản phẩm gỗ",
    "Giấy các loại": "Giấy các loại",
    "Bông": "Bông",
    "Sợi dệt": "Sợi dệt",
    "Vải": "Vải",
    "Nguyên PL dệt, may, giày dép": "Nguyên PL dệt, may, giày dép",
    "Thủy tinh và các SP từ thủy tinh": "Thủy tinh và các SP từ thủy tinh",
    "Phế liệu sắt thép": "Phế liệu sắt thép",
    "Sắt thép": "Sắt thép",
    "Sản phẩm từ sắt thép": "Sản phẩm từ sắt thép",
    "Kim loại thường khác": "Kim loại thường khác",
    "SP từ kim loại thường khác": "SP từ kim loại thường khác",
    "Điện tử, máy tính và linh kiện": "Điện tử, máy tính và linh kiện",
    "Hàng điện gia dụng và linh kiện": "Hàng điện gia dụng và linh kiện",
    "Điện thoại và linh kiện": "Điện thoại và linh kiện",
    "Máy ảnh, máy quay phim và linh kiện": "Máy ảnh, máy quay phim và linh kiện",
    "Máy móc thiết bị, dụng cụ PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Dây điện và cáp điện": "Dây điện và cáp điện",
    "Ô tô": "Ô tô",
    "Trong đó: Nguyên chiếc(*)": "Trong đó: Nguyên chiếc(*)",
    "Máy móc thiết bị, DC PT khác": "Máy móc thiết bị, dụng cụ PT khác",
    "Điện tử, máy tính và LK": "Điện tử, máy tính và linh kiện",
    "Hàng điện gia dụng và LK": "Hàng điện gia dụng và linh kiện",
    "Máy ảnh, máy quay phim và LK": "Máy ảnh, máy quay phim và linh kiện",
    "Thức ăn gia súc và nguyên phụ liệu": "Thức ăn gia súc và NPL",
    "Máy móc thiết bị, dụng cụ phụ tùng khác": "Máy móc thiết bị, dụng cụ PT khác",
    "SP nội thất từ chất liệu khác gỗ": "SP nội thất từ chất liệu khác gỗ",
    "Thủy tinh và cácSP từ thủy tinh": "Thủy tinh và các SP từ thủy tinh",
    "Điện thoại và LK": "Điện thoại và linh kiện",
    "Thủy tinh và các sản phẩm từ thủy tinh": "Thủy tinh và các SP từ thủy tinh",
    "Lúa mỳ": "Lúa mỳ",
    "Sản phẩm từ kim loại thường khác": "SP từ kim loại thường khác",
    "Phương tiện vận tải khác và PT": "Phương tiện vận tải khác và PT",
    "TỔNG TRỊ GIÁ (*)": "TỔNG TRỊ GIÁ",
    "Trong đó: Nguyên chiếc(**)": "Trong đó: Nguyên chiếc(*)",
    "Dầu mỡ động thực vật": "Dầu mỡ động thực vật",
    "Sản phẩm khác từ dầu mỏ": "Sản phẩm khác từ dầu mỏ",
    "Thuốc trừ sâu": "Thuốc trừ sâu",
    "Xe máy và linh kiện, phụ tùng": "Xe máy và linh kiện, phụ tùng",
    "Máy móc thiết bị, DC PT": "Máy móc thiết bị, dụng cụ PT khác",
    "Máy móc thiết bị, dụng cụ PT": "Máy móc thiết bị, dụng cụ PT khác",
    "Điện thoại các loại và LK": "Điện thoại và linh kiện",
    "Máy móc thiết bị, DC, PT khác": "Máy móc thiết bị, dụng cụ PT khác",
}

dfs_nk["Ngành"] = dfs_nk["Ngành"].map(convert_nganh)
dfs_nk

,Ngành,Total-M,Total-YOY,YOY,YTD-YOY,Date
0,TỔNG TRỊ GIÁ,31080,84980,9.70548,13.883591,2024-03-29
1,Khu vực kinh tế trong nước,10830,29696.990639,2.912738,14.393183,2024-03-29
2,Khu vực có vốn đầu tư NN,20250,55283.009361,13.719834,13.611718,2024-03-29
3,Thủy sản,230,636.436115,-6.206037,-3.933351,2024-03-29
4,Sữa và sản phẩm sữa,90,236.389268,-26.184634,-25.339836,2024-03-29
...,...,...,...,...,...,...
2477,"Máy móc thiết bị, dụng cụ PT khác",3000,3000,3.786542,3.786542,2019-01-29
2478,Ô tô,597.929161,597.929161,118.872049,118.872049,2019-01-29
2479,Trong đó: Nguyên chiếc(*),247.929161,247.929161,1263.290828,1263.290828,2019-01-29
2480,"Xe máy và linh kiện, phụ tùng",75,75,53.731541,53.731541,2019-01-29


In [117]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_nk[
    (
        (dfs_nk["Ngành"] == "TỔNG TRỊ GIÁ")
        | (dfs_nk["Ngành"] == "Khu vực kinh tế trong nước")
        | (dfs_nk["Ngành"] == "Khu vực có vốn đầu tư NN")
    )
    & (dfs_nk["Date"] > "2015-01-01")
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

/home/hien/.pyenv/versions/3.9.16/envs/vn-macroeconomics/lib/python3.9/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

